In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!ngrok authtoken '2G9vdCAb1Cbq9ARxqciS6hyJBZC_oNZNGBNeKkEnCBpWDqD'
!pip install flask-cors


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=16ae9b01be9f2312b095a09562aa676e537cff42e6b1748f8a4a39ed4a757e03
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

import os
import cv2
import numpy as np
# load and evaluate a saved model
from numpy import loadtxt
from tensorflow.keras.models import load_model
import base64
from PIL import Image
from io import BytesIO

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

DRIVE_LOCATION = "/content/gdrive/MyDrive"
DATADIR = f"{DRIVE_LOCATION}/testing"

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)


def upload_image(request_data):
  image_string = request_data['image'].split(",")[-1]
  patient_name = request_data['name']
  patient_dir = f"{DATADIR}/{patient_name}"

  if not os.path.exists(patient_dir):
    os.makedirs(patient_dir)

  im = Image.open(BytesIO(base64.b64decode(image_string))).convert("RGB")
  im.save(f"{DATADIR}/{patient_name}/{patient_name}.jpg", 'JPEG')
  
@app.route("/predict", methods=['GET', 'POST'])
def predict():
    request_data = request.get_json()
    patient_name = request_data['name']
    patient_dir = f"{DATADIR}/{patient_name}"
    issue_type = request_data['issue'].lower()
    upload_image(request_data)

    hair_class_dict = {'AcneKeloidalis': 0,
                  'AlopeciaAreata': 1,
                  'Dandruff': 2,
                  'Healthyscalp': 3}
    hair_medicines = ["Cyclomethicone NF-5, UTMF Ceramide III, Dermaxyl, Polydimethyl Siloxane, Silicone Dioxide, Elastomer – 10, Antron(220), Vitamin C. – ( HEXILAK Ultra) – (Flattens, Lightens and Softens Scars).",
                      "Propylene Glycol, Aqua, Water (and) Vitis Vinifera Meristem Fruit Cell Culture (and) Hydrolysed Eruca Sativa Leaf (and) Hydrolysed Walnut Extract, Pyrus Malus Extract, Phenoxyethanol (and) Ethylhexylglycerin, Niacinamide, Camellia Sinensis Leaf Extract (and) Propylene Glycol (and) Water (and) Sorbitol (and) Citric Acid (and) Sodium Hydroxide (and) Ascorbic Acid, Nigella Sativa Extract, Eclipta Alba Extract, Panthenol, Biotin, Disodium EDTA. (PROANAGEN SOLUTION) (WOMEN) + Follihair Tablets.",
                      "Selenium Sulfide 2.5% w/v , Buffered emulsion base q.s (Selsun lotion)","NA"]
    
    direction_to_use_hair = ["Apply the cream in the affected area outer to inner area. Wash and dry the affected place before applying the cream.",
                        "Take 1 ml of the solution and apply it in the affected area massage gently for a minute or two.",
                        "Wash hair with mild soap. Rinse. Massage Selsun onto scalp to form lather. Leave on scalp for two to three minutes. Rinse. Repeat the procedure and rinse thoroughly. Use twice weekly",
                        "NA"]

    skin_class_dict = {'Corn_callus': 0,
                      'Tiniea ': 1,
                      'Warts': 2,
                      'Keloid': 3,
                      'Pityriasisversicolor': 4,
                      'NO ISSUES FOUND':5}
    skin_medicines = ["Salicylic acid 17% w/w (Without infection)Heel pads and insoles Salicylic acid 17% w/w + Amoxycillin and Potassium Clavulanate (With Infection)",
                      "Luliconazole IP 1.0% w/w, Benzyl Alcohol IP 1.0% w/w (Lulitec Cream) Chlorohydrate 20% w/w, Aqua, Aluminum Chlorohydrate, Propylene Glycol, PPG-15 Stearyl Ether, Cetearyl Alcohol, Glyceryl Stearate (and) PEG-100 Stearate, Urea, Laureth-23, Dimethicone, SodiumLauryl Sulfate, DMDM Hydantoin, Methylchloroisothiazolinone (and) Methylisothiazolinone, Disodium EDTA, Perfume. (Seba Med)",
                      "Salicylic Acid 16.7%W/V Lactic Acid 16.7%W/V (DeWarts Cream) Most warts are harmless and clear up without treatment.The length of time it takes a wart to disappear will vary from person to person. It may take up to 2 years for the viral infection to leave your system and for the wart to disappear.You might decide to treat your wart if it is painful, or in an area that is causing discomfort or embarrassment.",
                      "Hydroquinone U.S.P.2% w/w, Tretinoin B.P. 0.025% w/w, Mometasone Furoate I.P.0.10% w/w In a cream base q.s. (No Scars)Octinoxate, Niacinamide, Glycolic Acid, Kojic Acid Dipalmitate, Arbutin, Mulberry Extract Tocopheryl Acetate, Allantoin, Licorice Extract & Tetrahydrocurcumin Gel ( Kojivit Ultra)",
                      "Cindamycin and Benzoyl Peroxide Gel (PEROCLIN 2.5%) Azelaic Acid Cream 20% w/w (Azobril Forte)",
                      "NA"]

    direction_to_use_skin = ["Wash the affected area and dry thoroughly apply one drop at a time with applicator, to sufficiently cover each corn/callus; let dry use cushion with hole centered over corn or callus to relieve pressure Repeat this procedure once or twice daily as needed for up to 14 days (until corn/callus is removed)",
                            "Wash the affected area and dry thoroughly apply a little Lulitec Cream from outer ring to inner center let this rest for 20 Mins, apply Seba Med on top the previously applied layer.",
                            "Wash and clean the affected area gently apply the cream on the affected area, Use 2 to 3 times a day",
                            "Wash and clean the affected area gently apply the cream on the affected area, Use 2 to 3 times a day",
                            "Wash the affected area and dry thoroughly apply a little PEROCLIN 2.5% Cream from outer ring to inner center let this rest for 20 Mins same for Azobril Forte. Apply very Gently on the affected area",
                            "NA"]

   
    class_names = {}
    class_values = []
    medicines = []
    direction_to_use = []
    model_name = ""
    doctor_details = {
        "imgUrl": "https://cdn.medimetry.com/production/wp-content/uploads/2017/01/22221806/Vignesh.jpg",
        "name": "Dr. Vignesh Kumar",
        "qualification": "MBBS, MD DVL"
    }

    if issue_type == "hair":
      class_names = list(hair_class_dict.keys())
      class_values = list(hair_class_dict.values())
      medicines = hair_medicines
      direction_to_use = direction_to_use_hair
      model_name = "hair_densenet_model2.h5"
    else:
      class_names = list(skin_class_dict.keys())
      class_values = list(skin_class_dict.values())
      medicines = skin_medicines
      direction_to_use = direction_to_use_skin
      model_name = "skin_densenet_model2.h5"
    
  
    for img in os.listdir(patient_dir):
            img_array = cv2.imread(os.path.join(patient_dir,img))
            img = (np.expand_dims(img_array,0))
            new_array = cv2.resize(img_array,(100,100))
            new_array = new_array/255
            ghh = new_array.reshape(-1,100,100, 3)
            pre_trained_model=load_model(f"{DRIVE_LOCATION}/Capstone/{model_name}")
            y_pred=pre_trained_model.predict(ghh)
            A = np.argmax(y_pred, axis=-1)
            response = {}
            for i in class_values:
              if class_values[i]==A[0]:
                response["predictedDisease"] = [class_names[i]]
                response["causes"] = []
                response["medicine"] = [medicines[i]]
                response["direction_to_use"] = [direction_to_use[i]]
                response["doctor"] = doctor_details
    
    return response
    
app.run()


Mounted at /content/gdrive
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0ee2-34-72-92-210.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [20/Oct/2022 08:00:48] "OPTIONS /predict HTTP/1.1" 200 -


1/1 [==============================] - 4s 4s/step


INFO:werkzeug:127.0.0.1 - - [20/Oct/2022 08:01:06] "POST /predict HTTP/1.1" 200 -
